In [ ]:
#Task A Qn:1

#Data Model and Justification:
    
1)In this assignment we are planning to implement Embedded model since the given dataset (climate_historic and hotspot_historic) appears to be small and has one to many relationship. Each climate data has multiple values in hotspot collection. Henceforth, we are sticking to Embedded hierarchy to represent the data
2)There are certain questions in the assignment which may need a join two tables to produce desired results. This is additional reason for choosing Embedded model. 

#Structure:
{
Object_Id:<value>
	Date:<date>
	Hotspot [
                {
				confidence : <value>
				latitude : <value>
				surface_temperature_celcius : <value>
				longitude : <value>
				datetime : <value>
                }
                {				
				confidence : <value>
				latitude : <value>
				surface_temperature_celcius : <value>
				longitude : <value>
				datetime : <value>
                }
          ]
                 Climate [
                            {
                                relative_humidity : <value>
                                precipitation : <value>
                                station : < value>
                                air_Temperature_celcius : <value> 
                                windspeed_knots : < value>
                                max_wind_speed : <value>	
                             }
                         ]
}

In [3]:
#Question Task B Qn:1
import csv
import json
from datetime import datetime

climateData=[]
climate_hotspot=[]
hotspotData=[]
hs=open('hotspot_historic.csv') #read hotspot file
reader=csv.DictReader(hs)
for data in reader:
    hotspotData.append(data) #convert into array
cs=open('climate_historic.csv') #read climate file
reader=csv.DictReader(cs)

for data in reader:
    climateData.append(data) #convert into array

for climate in climateData:
    climate_hotspot.append({})
position=0
for record in climate_hotspot:
    newDate=datetime.strptime(climateData[position]["date"], '%d/%m/%Y')  #date conversion
    record["date"] = newDate
    climateData[position].pop("date")
    for data in climateData[position]:
        try:
                if climateData[position][data].isdigit(): #checking for integer data
                    climateData[position][data] = int(climateData[position][data])
                elif float(climateData[position][data]): #checking for floating point data
                    climateData[position][data] = float(climateData[position][data])
        except:
                climateData[position][data] = climateData[position][data]
    record["climate"]=climateData[position]
    position+=1

for document in hotspotData:
    for data in document:
        try:
            if document[data].isdigit():
                document[data]=int(document[data])
            elif float(document[data]):
                document[data]=float(document[data])
        except:
            document[data]=document[data]
for hotSpotRecord in climate_hotspot:
    hotSpotRecord["hotspot"] = []
    hotSpotFinal={}
    for totalrecords in hotspot:
        hotSpotFinal=totalrecords.copy()
        if ((hotSpotRecord["date"])== datetime.strptime(totalrecords["date"],'%d/%m/%Y')):
            hotSpotFinal.pop("date")
            hotSpotRecord["hotspot"].append(hotSpotFinal)
print(climate_hotspot)

[{'date': datetime.datetime(2016, 12, 31, 0, 0), 'hotspot': [], 'climate': {'windspeed_knots': 7.9, 'air_temperature_celcius': 19, 'precipitation ': ' 0.00I', 'relative_humidity': 56.8, 'station': 948700, 'max_wind_speed': 11.1}}, {'date': datetime.datetime(2017, 1, 2, 0, 0), 'hotspot': [], 'climate': {'windspeed_knots': 9.2, 'air_temperature_celcius': 15, 'precipitation ': ' 0.02G', 'relative_humidity': 50.7, 'station': 948700, 'max_wind_speed': 13}}, {'date': datetime.datetime(2017, 1, 3, 0, 0), 'hotspot': [], 'climate': {'windspeed_knots': 8.1, 'air_temperature_celcius': 16, 'precipitation ': ' 0.00G', 'relative_humidity': 53.6, 'station': 948700, 'max_wind_speed': 15}}, {'date': datetime.datetime(2017, 1, 4, 0, 0), 'hotspot': [], 'climate': {'windspeed_knots': 7.7, 'air_temperature_celcius': 24, 'precipitation ': ' 0.00I', 'relative_humidity': 61.6, 'station': 948700, 'max_wind_speed': 14}}, {'date': datetime.datetime(2017, 1, 5, 0, 0), 'hotspot': [], 'climate': {'windspeed_knots':

In [4]:
#Task B Qn 1 continuation

#Inserting the merged result into mongodb collections
from pymongo import MongoClient
from pprint import pprint 
client = MongoClient()
db=client.fit5148_assignment_db
hotspot_climate_file=db.mergedData
for record in climate_hotspot:
    record_id = hotspot_climate_file.insert_one(record)
    print(record_id)


In [7]:
#Task B Qn:2a

#to identify climate data on 10/12/2017
task2a=hotspot_climate_file.find({"date":datetime.strptime("10/12/2017",'%d/%m/%Y')},{"climate":1,"_id":0})
for showClimate in task2a:
    pprint(showClimate)


{'climate': {'air_temperature_celcius': 17,
             'max_wind_speed': 14,
             'precipitation ': ' 0.00I',
             'relative_humidity': 53.5,
             'station': 948702,
             'windspeed_knots': 7.3}}


In [8]:
#Task B Qn:2b

#unwind the hotspot to deconstruct the array
#First I'm selecting surface temperature between 65 and 100
#Secondly,I'm displaying only latitude,longitude,surface temperature

task2b=hotspot_climate_file.aggregate([
                          {"$unwind":"$hotspot"},
                           {"$match":{"hotspot.surface_temperature_celcius":{"$gt":65,"$lt":100}}},
                            {"$project":{"_id":0,"hotspot.latitude":1,"hotspot.longitude":1,"hotspot.surface_temperature_celcius":1,"hotspot.confidence":1}}
                         ])
for displayclimate in task2b: 
    pprint(displayclimate)

{'hotspot': {'confidence': 94,
             'latitude': -37.2284,
             'longitude': 147.9187,
             'surface_temperature_celcius': 73}}
{'hotspot': {'confidence': 97,
             'latitude': -37.6572,
             'longitude': 142.0703,
             'surface_temperature_celcius': 80}}
{'hotspot': {'confidence': 84,
             'latitude': -37.0193,
             'longitude': 148.1459,
             'surface_temperature_celcius': 71}}
{'hotspot': {'confidence': 100,
             'latitude': -37.4229,
             'longitude': 147.027,
             'surface_temperature_celcius': 99}}
{'hotspot': {'confidence': 80,
             'latitude': -37.0055,
             'longitude': 148.1582,
             'surface_temperature_celcius': 68}}
{'hotspot': {'confidence': 85,
             'latitude': -37.4128,
             'longitude': 147.0242,
             'surface_temperature_celcius': 98}}
{'hotspot': {'confidence': 90,
             'latitude': -34.357,
             'longitude': 141

             'surface_temperature_celcius': 71}}
{'hotspot': {'confidence': 90,
             'latitude': -36.0387,
             'longitude': 145.6787,
             'surface_temperature_celcius': 67}}
{'hotspot': {'confidence': 94,
             'latitude': -36.6349,
             'longitude': 142.2125,
             'surface_temperature_celcius': 86}}
{'hotspot': {'confidence': 86,
             'latitude': -36.6986,
             'longitude': 142.7259,
             'surface_temperature_celcius': 72}}
{'hotspot': {'confidence': 94,
             'latitude': -38.0052,
             'longitude': 142.6179,
             'surface_temperature_celcius': 89}}
{'hotspot': {'confidence': 100,
             'latitude': -37.2168,
             'longitude': 147.4748,
             'surface_temperature_celcius': 89}}
{'hotspot': {'confidence': 96,
             'latitude': -37.222,
             'longitude': 147.4689,
             'surface_temperature_celcius': 78}}
{'hotspot': {'confidence': 92,
             '

             'surface_temperature_celcius': 75}}
{'hotspot': {'confidence': 100,
             'latitude': -37.4591,
             'longitude': 144.9492,
             'surface_temperature_celcius': 89}}
{'hotspot': {'confidence': 91,
             'latitude': -36.3487,
             'longitude': 145.5315,
             'surface_temperature_celcius': 68}}
{'hotspot': {'confidence': 90,
             'latitude': -36.5399,
             'longitude': 144.678,
             'surface_temperature_celcius': 66}}
{'hotspot': {'confidence': 97,
             'latitude': -36.8158,
             'longitude': 142.6229,
             'surface_temperature_celcius': 80}}
{'hotspot': {'confidence': 100,
             'latitude': -36.4518,
             'longitude': 144.7752,
             'surface_temperature_celcius': 97}}
{'hotspot': {'confidence': 90,
             'latitude': -36.7903,
             'longitude': 141.618,
             'surface_temperature_celcius': 67}}
{'hotspot': {'confidence': 97,
             '

             'surface_temperature_celcius': 70}}
{'hotspot': {'confidence': 100,
             'latitude': -37.5728,
             'longitude': 142.6348,
             'surface_temperature_celcius': 98}}
{'hotspot': {'confidence': 100,
             'latitude': -37.4352,
             'longitude': 143.1444,
             'surface_temperature_celcius': 96}}
{'hotspot': {'confidence': 92,
             'latitude': -36.5828,
             'longitude': 144.5775,
             'surface_temperature_celcius': 70}}
{'hotspot': {'confidence': 96,
             'latitude': -36.5987,
             'longitude': 144.4995,
             'surface_temperature_celcius': 79}}
{'hotspot': {'confidence': 98,
             'latitude': -36.5262,
             'longitude': 144.5737,
             'surface_temperature_celcius': 84}}
{'hotspot': {'confidence': 95,
             'latitude': -36.6995,
             'longitude': 142.978,
             'surface_temperature_celcius': 76}}
{'hotspot': {'confidence': 93,
             

             'longitude': 146.0602,
             'surface_temperature_celcius': 90}}
{'hotspot': {'confidence': 92,
             'latitude': -37.7889,
             'longitude': 143.1324,
             'surface_temperature_celcius': 71}}
{'hotspot': {'confidence': 93,
             'latitude': -36.2306,
             'longitude': 143.6408,
             'surface_temperature_celcius': 72}}
{'hotspot': {'confidence': 91,
             'latitude': -36.4168,
             'longitude': 141.6939,
             'surface_temperature_celcius': 68}}
{'hotspot': {'confidence': 97,
             'latitude': -36.4211,
             'longitude': 141.6875,
             'surface_temperature_celcius': 81}}
{'hotspot': {'confidence': 96,
             'latitude': -36.8094,
             'longitude': 142.8885,
             'surface_temperature_celcius': 77}}
{'hotspot': {'confidence': 84,
             'latitude': -37.8323,
             'longitude': 147.2232,
             'surface_temperature_celcius': 68}}
{'hotspot

             'latitude': -37.634,
             'longitude': 149.237,
             'surface_temperature_celcius': 71}}
{'hotspot': {'confidence': 98,
             'latitude': -37.605,
             'longitude': 149.302,
             'surface_temperature_celcius': 83}}
{'hotspot': {'confidence': 99,
             'latitude': -37.6,
             'longitude': 149.325,
             'surface_temperature_celcius': 86}}
{'hotspot': {'confidence': 95,
             'latitude': -37.618,
             'longitude': 149.281,
             'surface_temperature_celcius': 76}}
{'hotspot': {'confidence': 100,
             'latitude': -36.282,
             'longitude': 146.157,
             'surface_temperature_celcius': 71}}
{'hotspot': {'confidence': 92,
             'latitude': -37.95,
             'longitude': 142.366,
             'surface_temperature_celcius': 70}}
{'hotspot': {'confidence': 93,
             'latitude': -37.875,
             'longitude': 142.51,
             'surface_temperature_celciu

In [9]:
#Task B Qn:2c

#Printing the seletecd data on 16/12/2017 and 17/12/2017

task2c=hotspot_climate_file.aggregate([                         
                           {"$match":{"date":{"$gte":datetime.strptime("16/12/2017",'%d/%m/%Y'),"$lte":datetime.strptime("17/12/2017",'%d/%m/%Y')}}},
                            {"$project":{"_id":0,"date":1,"climate.air_temperature_celcius":1,"climate.relative_humidity":1,"climate.max_wind_speed":1,"hotspot.surface_temperature_celcius":1}}
                         ])

for display in task2c:
    pprint(display)
    

{'climate': {'air_temperature_celcius': 18,
             'max_wind_speed': 13,
             'relative_humidity': 53.7},
 'date': datetime.datetime(2017, 12, 16, 0, 0),
 'hotspot': [{'surface_temperature_celcius': 43},
             {'surface_temperature_celcius': 33},
             {'surface_temperature_celcius': 54},
             {'surface_temperature_celcius': 73},
             {'surface_temperature_celcius': 55},
             {'surface_temperature_celcius': 75},
             {'surface_temperature_celcius': 55},
             {'surface_temperature_celcius': 66},
             {'surface_temperature_celcius': 56},
             {'surface_temperature_celcius': 60},
             {'surface_temperature_celcius': 73},
             {'surface_temperature_celcius': 48},
             {'surface_temperature_celcius': 55},
             {'surface_temperature_celcius': 64},
             {'surface_temperature_celcius': 57}]}
{'climate': {'air_temperature_celcius': 21,
             'max_wind_speed': 13,
  

In [10]:
#Task B Qn:2d

#printing data with confidence greater than 80 and less than 100

task2d=hotspot_climate_file.aggregate([
                          {"$unwind":"$hotspot"},
                           {"$match":{"hotspot.confidence":{"$gt":80,"$lt":100}}},
                            {"$project":{"_id":0,"hotspot.datetime":1,"climate.air_temperature_celcius":1,"hotspot.surface_temperature_celcius":1,"hotspot.confidence":1}}
                         ])

for display in task2d:
    pprint(display)


{'climate': {'air_temperature_celcius': 20},
 'hotspot': {'confidence': 87,
             'datetime': '2017-03-06T05:06:30',
             'surface_temperature_celcius': 62}}
{'climate': {'air_temperature_celcius': 20},
 'hotspot': {'confidence': 85,
             'datetime': '2017-03-06T05:06:20',
             'surface_temperature_celcius': 59}}
{'climate': {'air_temperature_celcius': 19},
 'hotspot': {'confidence': 88,
             'datetime': '2017-03-07T04:16:10',
             'surface_temperature_celcius': 64}}
{'climate': {'air_temperature_celcius': 23},
 'hotspot': {'confidence': 86,
             'datetime': '2017-03-09T13:23:40',
             'surface_temperature_celcius': 41}}
{'climate': {'air_temperature_celcius': 19},
 'hotspot': {'confidence': 94,
             'datetime': '2017-03-10T04:45:30',
             'surface_temperature_celcius': 73}}
{'climate': {'air_temperature_celcius': 19},
 'hotspot': {'confidence': 97,
             'datetime': '2017-03-10T04:45:30',
           

{'climate': {'air_temperature_celcius': 14},
 'hotspot': {'confidence': 82,
             'datetime': '2017-03-31T04:58:30',
             'surface_temperature_celcius': 56}}
{'climate': {'air_temperature_celcius': 14},
 'hotspot': {'confidence': 86,
             'datetime': '2017-03-31T04:57:10',
             'surface_temperature_celcius': 61}}
{'climate': {'air_temperature_celcius': 14},
 'hotspot': {'confidence': 81,
             'datetime': '2017-03-31T00:36:30',
             'surface_temperature_celcius': 54}}
{'climate': {'air_temperature_celcius': 15},
 'hotspot': {'confidence': 91,
             'datetime': '2017-04-03T13:17:00',
             'surface_temperature_celcius': 43}}
{'climate': {'air_temperature_celcius': 15},
 'hotspot': {'confidence': 83,
             'datetime': '2017-04-03T13:15:30',
             'surface_temperature_celcius': 40}}
{'climate': {'air_temperature_celcius': 15},
 'hotspot': {'confidence': 96,
             'datetime': '2017-04-03T13:15:10',
           

             'datetime': '2017-04-04T04:33:50',
             'surface_temperature_celcius': 81}}
{'climate': {'air_temperature_celcius': 16},
 'hotspot': {'confidence': 81,
             'datetime': '2017-04-04T04:33:40',
             'surface_temperature_celcius': 54}}
{'climate': {'air_temperature_celcius': 16},
 'hotspot': {'confidence': 85,
             'datetime': '2017-04-04T04:33:30',
             'surface_temperature_celcius': 59}}
{'climate': {'air_temperature_celcius': 16},
 'hotspot': {'confidence': 82,
             'datetime': '2017-04-04T04:33:10',
             'surface_temperature_celcius': 62}}
{'climate': {'air_temperature_celcius': 16},
 'hotspot': {'confidence': 82,
             'datetime': '2017-04-04T04:33:10',
             'surface_temperature_celcius': 62}}
{'climate': {'air_temperature_celcius': 16},
 'hotspot': {'confidence': 97,
             'datetime': '2017-04-04T04:33:00',
             'surface_temperature_celcius': 79}}
{'climate': {'air_temperature_celcius'

             'surface_temperature_celcius': 62}}
{'climate': {'air_temperature_celcius': 19},
 'hotspot': {'confidence': 86,
             'datetime': '2017-04-06T04:20:40',
             'surface_temperature_celcius': 72}}
{'climate': {'air_temperature_celcius': 19},
 'hotspot': {'confidence': 81,
             'datetime': '2017-04-06T04:20:40',
             'surface_temperature_celcius': 57}}
{'climate': {'air_temperature_celcius': 19},
 'hotspot': {'confidence': 94,
             'datetime': '2017-04-06T04:20:40',
             'surface_temperature_celcius': 89}}
{'climate': {'air_temperature_celcius': 21},
 'hotspot': {'confidence': 85,
             'datetime': '2017-04-07T12:56:50',
             'surface_temperature_celcius': 41}}
{'climate': {'air_temperature_celcius': 21},
 'hotspot': {'confidence': 94,
             'datetime': '2017-04-07T12:56:30',
             'surface_temperature_celcius': 49}}
{'climate': {'air_temperature_celcius': 21},
 'hotspot': {'confidence': 86,
          

 'hotspot': {'confidence': 90,
             'datetime': '2017-04-13T04:33:30',
             'surface_temperature_celcius': 66}}
{'climate': {'air_temperature_celcius': 16},
 'hotspot': {'confidence': 89,
             'datetime': '2017-04-13T04:33:20',
             'surface_temperature_celcius': 66}}
{'climate': {'air_temperature_celcius': 16},
 'hotspot': {'confidence': 97,
             'datetime': '2017-04-13T04:33:10',
             'surface_temperature_celcius': 80}}
{'climate': {'air_temperature_celcius': 16},
 'hotspot': {'confidence': 88,
             'datetime': '2017-04-13T04:33:00',
             'surface_temperature_celcius': 63}}
{'climate': {'air_temperature_celcius': 16},
 'hotspot': {'confidence': 82,
             'datetime': '2017-04-13T04:33:00',
             'surface_temperature_celcius': 55}}
{'climate': {'air_temperature_celcius': 16},
 'hotspot': {'confidence': 98,
             'datetime': '2017-04-13T04:31:00',
             'surface_temperature_celcius': 82}}
{'clima

...

             'surface_temperature_celcius': 79}}
{'climate': {'air_temperature_celcius': 15},
 'hotspot': {'confidence': 83,
             'datetime': '2017-04-18T04:44:50',
             'surface_temperature_celcius': 56}}
{'climate': {'air_temperature_celcius': 15},
 'hotspot': {'confidence': 81,
             'datetime': '2017-04-18T04:44:50',
             'surface_temperature_celcius': 54}}
{'climate': {'air_temperature_celcius': 15},
 'hotspot': {'confidence': 98,
             'datetime': '2017-04-18T04:44:50',
             'surface_temperature_celcius': 84}}
{'climate': {'air_temperature_celcius': 15},
 'hotspot': {'confidence': 82,
             'datetime': '2017-04-18T04:44:50',
             'surface_temperature_celcius': 56}}
{'climate': {'air_temperature_celcius': 15},
 'hotspot': {'confidence': 82,
             'datetime': '2017-04-18T04:44:50',
             'surface_temperature_celcius': 56}}
{'climate': {'air_temperature_celcius': 15},
 'hotspot': {'confidence': 95,
          

{'climate': {'air_temperature_celcius': 10},
 'hotspot': {'confidence': 92,
             'datetime': '2017-05-04T04:53:10',
             'surface_temperature_celcius': 71}}
{'climate': {'air_temperature_celcius': 10},
 'hotspot': {'confidence': 94,
             'datetime': '2017-05-04T04:48:50',
             'surface_temperature_celcius': 73}}
{'climate': {'air_temperature_celcius': 10},
 'hotspot': {'confidence': 95,
             'datetime': '2017-05-04T04:48:20',
             'surface_temperature_celcius': 76}}
{'climate': {'air_temperature_celcius': 10},
 'hotspot': {'confidence': 87,
             'datetime': '2017-05-04T04:48:10',
             'surface_temperature_celcius': 62}}
{'climate': {'air_temperature_celcius': 10},
 'hotspot': {'confidence': 87,
             'datetime': '2017-05-04T04:46:50',
             'surface_temperature_celcius': 63}}
{'climate': {'air_temperature_celcius': 10},
 'hotspot': {'confidence': 81,
             'datetime': '2017-05-04T04:46:30',
           

{'climate': {'air_temperature_celcius': 14},
 'hotspot': {'confidence': 97,
             'datetime': '2017-05-05T03:50:20',
             'surface_temperature_celcius': 80}}
{'climate': {'air_temperature_celcius': 14},
 'hotspot': {'confidence': 96,
             'datetime': '2017-05-05T03:50:20',
             'surface_temperature_celcius': 78}}
{'climate': {'air_temperature_celcius': 12},
 'hotspot': {'confidence': 91,
             'datetime': '2017-05-06T04:39:00',
             'surface_temperature_celcius': 68}}
{'climate': {'air_temperature_celcius': 12},
 'hotspot': {'confidence': 95,
             'datetime': '2017-05-06T04:33:00',
             'surface_temperature_celcius': 76}}
{'climate': {'air_temperature_celcius': 12},
 'hotspot': {'confidence': 96,
             'datetime': '2017-05-06T04:32:50',
             'surface_temperature_celcius': 78}}
{'climate': {'air_temperature_celcius': 12},
 'hotspot': {'confidence': 85,
             'datetime': '2017-05-06T04:32:30',
           

             'surface_temperature_celcius': 62}}
{'climate': {'air_temperature_celcius': 10},
 'hotspot': {'confidence': 86,
             'datetime': '2017-05-15T04:30:00',
             'surface_temperature_celcius': 60}}
{'climate': {'air_temperature_celcius': 10},
 'hotspot': {'confidence': 83,
             'datetime': '2017-05-15T04:29:50',
             'surface_temperature_celcius': 63}}
{'climate': {'air_temperature_celcius': 10},
 'hotspot': {'confidence': 84,
             'datetime': '2017-05-15T04:29:50',
             'surface_temperature_celcius': 58}}
{'climate': {'air_temperature_celcius': 10},
 'hotspot': {'confidence': 87,
             'datetime': '2017-05-15T04:28:00',
             'surface_temperature_celcius': 62}}
{'climate': {'air_temperature_celcius': 10},
 'hotspot': {'confidence': 90,
             'datetime': '2017-05-15T04:27:50',
             'surface_temperature_celcius': 66}}
{'climate': {'air_temperature_celcius': 10},
 'hotspot': {'confidence': 84,
          

             'datetime': '2017-10-03T03:58:15',
             'surface_temperature_celcius': 63}}
{'climate': {'air_temperature_celcius': 13},
 'hotspot': {'confidence': 88,
             'datetime': '2017-10-03T03:58:14',
             'surface_temperature_celcius': 65}}
{'climate': {'air_temperature_celcius': 13},
 'hotspot': {'confidence': 87,
             'datetime': '2017-10-03T03:58:14',
             'surface_temperature_celcius': 59}}
{'climate': {'air_temperature_celcius': 19},
 'hotspot': {'confidence': 85,
             'datetime': '2017-10-04T04:41:00',
             'surface_temperature_celcius': 54}}
{'climate': {'air_temperature_celcius': 19},
 'hotspot': {'confidence': 88,
             'datetime': '2017-10-04T04:41:00',
             'surface_temperature_celcius': 58}}
{'climate': {'air_temperature_celcius': 17},
 'hotspot': {'confidence': 91,
             'datetime': '2017-10-07T12:59:33',
             'surface_temperature_celcius': 42}}
{'climate': {'air_temperature_celcius'

In [12]:
#Task B Qn:2e

#Printing top 10 surface temperature using $sort funtion. -1 is to print them in descending order

task2e=hotspot_climate_file.aggregate([
                          {"$unwind":"$hotspot"},
                        {"$sort":{"hotspot.surface_temperature_celcius" : -1}},
                        {"$project":{"_id":0}},
                        {"$limit":10}
                         ])

for display in task2e:
    pprint(display)

{'climate': {'air_temperature_celcius': 15,
             'max_wind_speed': 9.9,
             'precipitation ': ' 0.00I',
             'relative_humidity': 56.1,
             'station': 948701,
             'windspeed_knots': 5.1},
 'date': datetime.datetime(2017, 4, 18, 0, 0),
 'hotspot': {'confidence': 100,
             'datetime': '2017-04-18T04:52:00',
             'latitude': -38.1665,
             'longitude': 143.062,
             'surface_temperature_celcius': 124}}
{'climate': {'air_temperature_celcius': 16,
             'max_wind_speed': 12,
             'precipitation ': ' 0.00I',
             'relative_humidity': 47.5,
             'station': 948701,
             'windspeed_knots': 5.4},
 'date': datetime.datetime(2017, 4, 4, 0, 0),
 'hotspot': {'confidence': 100,
             'datetime': '2017-04-04T04:32:50',
             'latitude': -36.343,
             'longitude': 142.1986,
             'surface_temperature_celcius': 123}}
{'climate': {'air_temperature_celcius': 14,
  

In [13]:
#Task Qn:2f
#Certain fire count is zero,because hotspot data found is not found for that particular dates. 

task2f=hotspot_climate_file.aggregate([
                          {"$project":{"_id":0,"date":1,"numberOfFires":{"$size":"$hotspot"}}}
                         ])

for display in task2f:
    pprint(display)

{'date': datetime.datetime(2016, 12, 31, 0, 0), 'numberOfFires': 0}
{'date': datetime.datetime(2017, 1, 2, 0, 0), 'numberOfFires': 0}
{'date': datetime.datetime(2017, 1, 3, 0, 0), 'numberOfFires': 0}
{'date': datetime.datetime(2017, 1, 4, 0, 0), 'numberOfFires': 0}
{'date': datetime.datetime(2017, 1, 5, 0, 0), 'numberOfFires': 0}
{'date': datetime.datetime(2017, 1, 6, 0, 0), 'numberOfFires': 0}
{'date': datetime.datetime(2017, 1, 7, 0, 0), 'numberOfFires': 0}
{'date': datetime.datetime(2017, 1, 8, 0, 0), 'numberOfFires': 0}
{'date': datetime.datetime(2017, 1, 9, 0, 0), 'numberOfFires': 0}
{'date': datetime.datetime(2017, 1, 10, 0, 0), 'numberOfFires': 0}
{'date': datetime.datetime(2017, 1, 11, 0, 0), 'numberOfFires': 0}
{'date': datetime.datetime(2017, 1, 12, 0, 0), 'numberOfFires': 0}
{'date': datetime.datetime(2017, 1, 13, 0, 0), 'numberOfFires': 0}
{'date': datetime.datetime(2017, 1, 14, 0, 0), 'numberOfFires': 0}
{'date': datetime.datetime(2017, 1, 15, 0, 0), 'numberOfFires': 0}
{'

In [15]:
#Task Qn:2g
#for certain dates average is null because data is not available in the given dataset
task2g=hotspot_climate_file.aggregate([
                          {"$project":{"_id":0,"date":1,"AverageSurfaceTemperature":{"$avg":"$hotspot.surface_temperature_celcius"}}}
                         ])

for avgSurface in task2g:
    pprint(avgSurface)

{'AverageSurfaceTemperature': None,
 'date': datetime.datetime(2016, 12, 31, 0, 0)}
{'AverageSurfaceTemperature': None, 'date': datetime.datetime(2017, 1, 2, 0, 0)}
{'AverageSurfaceTemperature': None, 'date': datetime.datetime(2017, 1, 3, 0, 0)}
{'AverageSurfaceTemperature': None, 'date': datetime.datetime(2017, 1, 4, 0, 0)}
{'AverageSurfaceTemperature': None, 'date': datetime.datetime(2017, 1, 5, 0, 0)}
{'AverageSurfaceTemperature': None, 'date': datetime.datetime(2017, 1, 6, 0, 0)}
{'AverageSurfaceTemperature': None, 'date': datetime.datetime(2017, 1, 7, 0, 0)}
{'AverageSurfaceTemperature': None, 'date': datetime.datetime(2017, 1, 8, 0, 0)}
{'AverageSurfaceTemperature': None, 'date': datetime.datetime(2017, 1, 9, 0, 0)}
{'AverageSurfaceTemperature': None,
 'date': datetime.datetime(2017, 1, 10, 0, 0)}
{'AverageSurfaceTemperature': None,
 'date': datetime.datetime(2017, 1, 11, 0, 0)}
{'AverageSurfaceTemperature': None,
 'date': datetime.datetime(2017, 1, 12, 0, 0)}
{'AverageSurfaceTem

 'date': datetime.datetime(2017, 5, 10, 0, 0)}
{'AverageSurfaceTemperature': 59.36842105263158,
 'date': datetime.datetime(2017, 5, 11, 0, 0)}
{'AverageSurfaceTemperature': 51.5,
 'date': datetime.datetime(2017, 5, 12, 0, 0)}
{'AverageSurfaceTemperature': 58.611111111111114,
 'date': datetime.datetime(2017, 5, 13, 0, 0)}
{'AverageSurfaceTemperature': 49.0,
 'date': datetime.datetime(2017, 5, 14, 0, 0)}
{'AverageSurfaceTemperature': 53.950980392156865,
 'date': datetime.datetime(2017, 5, 15, 0, 0)}
{'AverageSurfaceTemperature': 39.666666666666664,
 'date': datetime.datetime(2017, 5, 16, 0, 0)}
{'AverageSurfaceTemperature': 52.0,
 'date': datetime.datetime(2017, 5, 17, 0, 0)}
{'AverageSurfaceTemperature': 44.142857142857146,
 'date': datetime.datetime(2017, 5, 18, 0, 0)}
{'AverageSurfaceTemperature': None,
 'date': datetime.datetime(2017, 5, 19, 0, 0)}
{'AverageSurfaceTemperature': None,
 'date': datetime.datetime(2017, 5, 20, 0, 0)}
{'AverageSurfaceTemperature': None,
 'date': datetime.

{'AverageSurfaceTemperature': None, 'date': datetime.datetime(2017, 9, 1, 0, 0)}
{'AverageSurfaceTemperature': None, 'date': datetime.datetime(2017, 9, 2, 0, 0)}
{'AverageSurfaceTemperature': None, 'date': datetime.datetime(2017, 9, 3, 0, 0)}
{'AverageSurfaceTemperature': None, 'date': datetime.datetime(2017, 9, 4, 0, 0)}
{'AverageSurfaceTemperature': None, 'date': datetime.datetime(2017, 9, 5, 0, 0)}
{'AverageSurfaceTemperature': None, 'date': datetime.datetime(2017, 9, 6, 0, 0)}
{'AverageSurfaceTemperature': None, 'date': datetime.datetime(2017, 9, 7, 0, 0)}
{'AverageSurfaceTemperature': None, 'date': datetime.datetime(2017, 9, 8, 0, 0)}
{'AverageSurfaceTemperature': None, 'date': datetime.datetime(2017, 9, 9, 0, 0)}
{'AverageSurfaceTemperature': 56.5,
 'date': datetime.datetime(2017, 9, 10, 0, 0)}
{'AverageSurfaceTemperature': None,
 'date': datetime.datetime(2017, 9, 11, 0, 0)}
{'AverageSurfaceTemperature': None,
 'date': datetime.datetime(2017, 9, 12, 0, 0)}
{'AverageSurfaceTemper

{'AverageSurfaceTemperature': None,
 'date': datetime.datetime(2017, 12, 20, 0, 0)}
{'AverageSurfaceTemperature': 46.0,
 'date': datetime.datetime(2017, 12, 21, 0, 0)}
{'AverageSurfaceTemperature': None,
 'date': datetime.datetime(2017, 12, 22, 0, 0)}
{'AverageSurfaceTemperature': None,
 'date': datetime.datetime(2017, 12, 23, 0, 0)}
{'AverageSurfaceTemperature': 32.0,
 'date': datetime.datetime(2017, 12, 24, 0, 0)}
{'AverageSurfaceTemperature': 54.0,
 'date': datetime.datetime(2017, 12, 25, 0, 0)}
{'AverageSurfaceTemperature': None,
 'date': datetime.datetime(2017, 12, 26, 0, 0)}
{'AverageSurfaceTemperature': 62.75,
 'date': datetime.datetime(2017, 12, 27, 0, 0)}
{'AverageSurfaceTemperature': None,
 'date': datetime.datetime(2017, 12, 28, 0, 0)}
{'AverageSurfaceTemperature': None,
 'date': datetime.datetime(2017, 12, 29, 0, 0)}
{'AverageSurfaceTemperature': None,
 'date': datetime.datetime(2017, 12, 30, 0, 0)}
{'AverageSurfaceTemperature': None,
 'date': datetime.datetime(2017, 12, 31